In [4]:
from utils import h36motion3d as datasets
from torch.utils.data import DataLoader
from torch import nn
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import torch.optim as optim
import torch.autograd
import torch
import numpy as np
from utils.loss_funcs import *
from utils.data_utils import define_actions
from utils.h36_3d_viz import visualize
import time
import math

import torch.nn.functional as F

#reproducibility stuff
torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)

torch.backends.cudnn.deterministic=True

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: %s'%device)

Using device: cpu


In [5]:
class Attention(nn.Module):
    def __init__(self, attn_dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(attn_dropout)

    def forward(self, query, key, value, mask=None):
        d_k = key.size(-1) #-1 refers to the last dimension
        # Compute the dot product of the query and key, and scale it
        #attn = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k) #Note: Transpose(-2, -1) swaps the second last dimension with the last dimension
        attn = torch.einsum('BHLD, BHMD -> BHLM', query, key) / math.sqrt(d_k)

        if mask is not None:
            attn = attn.masked_fill(mask == 0, -1e9)

        attn = self.dropout(F.softmax(value, dim=-1))
        
        #output = torch.matmul(attn, value)
        output = torch.einsum('BHLM, BHLD -> BHLD', attn, value)

        return output, attn
